In [1]:
from tkinter import *
import mysql.connector as mysql
from tkinter import messagebox
from PIL import ImageTk, Image

def main():
    root = Tk()

    # MySQL connection with exception handling
    try:
        conn = mysql.connect(user='root', password='', host='localhost', db='ola')
        cursor = conn.cursor()
    except mysql.Error as err:
        messagebox.showerror("Database Error", f"Error connecting to database: {err}")
        return

    root.geometry("500x500")
    root.title("Ola Ride Booking App")

    # Load logo image
    try:
        img = Image.open("ola.png")  # Load the image
        img = img.resize((150, 150))  # Resize the image
        logo = ImageTk.PhotoImage(img)
    except Exception as e:
        messagebox.showerror("Image Error", f"Could not load image: {e}")
        return

    # Add the logo to the main window
    logo_label = Label(root, image=logo)
    logo_label.image = logo  # Keep a reference to avoid garbage collection
    logo_label.place(x=175, y=20)  # Center the logo

    def register():
        root.destroy()
        reg = Tk()
        reg.geometry("500x500")
        reg.title("Register")

        # Add the same logo to the registration window
        try:
            reg.logo = ImageTk.PhotoImage(img)  # Store a reference to avoid garbage collection
            logo_label = Label(reg, image=reg.logo)
            logo_label.place(x=175, y=20)
        except Exception as e:
            messagebox.showerror("Image Error", f"Could not load image: {e}")

        l1 = Label(reg, text="=======User Registration=====", font=("Arial", 20))
        l2 = Label(reg, text="Username")
        l3 = Label(reg, text="Email ID")
        l4 = Label(reg, text="Password")

        e1 = Entry(reg)
        e2 = Entry(reg)
        e3 = Entry(reg, show="*")

        l1.place(x=50, y=180)  # Adjusted placement
        l2.place(x=100, y=250)
        e1.place(x=200, y=250)
        l3.place(x=100, y=280)
        e2.place(x=200, y=280)
        l4.place(x=100, y=310)
        e3.place(x=200, y=310)

        def reguser():
            sql = "INSERT INTO users(username, email, password) VALUES (%s, %s, %s)"
            val = (e1.get(), e2.get(), e3.get())
            cursor.execute(sql, val)
            conn.commit()
            messagebox.showinfo("Confirmation", "User Registered Successfully!")

        bt1 = Button(reg, text="Register", command=reguser)
        bt1.place(x=100, y=350)

        def logcl():
            reg.destroy()
            main()

        bt2 = Button(reg, text="Go Back", command=logcl)
        bt2.place(x=100, y=380)

        reg.mainloop()

    def login():
        root.destroy()
        reg2 = Tk()
        reg2.title("Login")
        l1 = Label(reg2, text="======User Login=====", font=("Arial", 20))
        l1.place(x=50, y=20)
        l2 = Label(reg2, text="Username/Email")
        l2.place(x=100, y=100)
        l3 = Label(reg2, text="Password")
        l3.place(x=100, y=130)
        e1 = Entry(reg2)
        e1.place(x=200, y=100)
        e2 = Entry(reg2, show="*")
        e2.place(x=200, y=130)

        def book_ride(username):
            reg2.destroy()
            ride = Tk()
            ride.title("Book a Ride")

            l1 = Label(ride, text="Enter Pickup Location")
            l1.place(x=100, y=100)
            l2 = Label(ride, text="Enter Dropoff Location")
            l2.place(x=100, y=130)

            pickup = Entry(ride)
            pickup.place(x=250, y=100)
            dropoff = Entry(ride)
            dropoff.place(x=250, y=130)

            def confirm_ride():
                try:
                    # Get user ID from the 'users' table
                    cursor.execute("SELECT id FROM users WHERE username=%s OR email=%s", (username, username))
                    user_id = cursor.fetchone()

                    if user_id:
                        user_id = user_id[0]  # Extract the user ID from the result tuple

                        # Close previous result set and commit
                        cursor.fetchall()  # Ensure that no results are left unread
                        conn.commit()

                        # Now insert the ride details
                        sql = "INSERT INTO rides(user_id, pickup, dropoff) VALUES (%s, %s, %s)"
                        val = (user_id, pickup.get(), dropoff.get())
                        cursor.execute(sql, val)
                        conn.commit()  # Commit the transaction

                        messagebox.showinfo("Confirmation", f"Ride from {pickup.get()} to {dropoff.get()} is booked.")
                        ride.destroy()
                        main()
                    else:
                        messagebox.showerror("Error", "User ID not found.")
                except mysql.Error as err:
                    messagebox.showerror("Database Error", f"Error: {err}")

            bt = Button(ride, text="Confirm Ride", command=confirm_ride)
            bt.place(x=200, y=160)

            ride.geometry("500x500")
            ride.mainloop()

        def loguser():
            q = "SELECT * FROM users WHERE (username=%s OR email=%s) AND password=%s"
            cursor.execute(q, (e1.get(), e1.get(), e2.get()))
            result = cursor.fetchone()
            if result:
                username = e1.get()  # Store username or email
                book_ride(username)  # Pass it to the next window
            else:
                messagebox.showerror("Error", "Invalid Credentials")

        bt1 = Button(reg2, text="Login", command=loguser)
        bt1.place(x=100, y=170)

        def regcl():
            reg2.destroy()
            main()

        bt2 = Button(reg2, text="Go Back", command=regcl)
        bt2.place(x=100, y=210)

        reg2.geometry("500x500")
        reg2.mainloop()

    b1 = Button(root, text="Register", command=register)
    b1.place(x=100, y=240)
    b2 = Button(root, text="Login", command=login)
    b2.place(x=100, y=270)

    root.mainloop()

main()
